In [1]:
include("../src/TemporalInstanton.jl")
using TemporalInstanton

LoadError: LoadError: LoadError: LoadError: syntax: invalid interpolation syntax: "$$"
while loading /home/jkersulis/Documents/Dropbox/InstantonAnew/Jonas Edits/Time-Coupled Instanton/temporal-instanton/src/solvetmpinst.jl, in expression starting on line 4
while loading /home/jkersulis/Documents/Dropbox/InstantonAnew/Jonas Edits/Time-Coupled Instanton/temporal-instanton/src/TemporalInstanton.jl, in expression starting on line 27
while loading In[1], in expression starting on line 1

*hello\*

In [2]:
"""
    expand_renewable_vector(x,Ridx,N,T) -> P
Starting with `x`, a `length(Ridx)*T` vector containing generation forecast for all nodes and time steps, insert zeros to produce an `N*T`-long vector.
"""
function expand_renewable_vector(x,Ridx,N,T)
    idx = Array(Integer,0)
    for i = 0:T-1
        append!(idx,Ridx + i*N)
    end
    P = zeros(N*T)
    P[idx] = x
    return P
end

expand_renewable_vector (generic function with 1 method)

In [6]:
"""
    fixed_wind_A(T,Y,ref,k) -> A
Generate the power balance constraint A matrix from problem dimensions, admittance matrix, and generator participation factors. Use this function when variable (renewable) injections are known, to run power flow and determine angle differences throughout the network.

Assumes the admittance matrix is n-by-n.

Returns A, which is (n+1)T-by-(nr+n+1)T

* `nr` is the number of wind farms in the network
* `n` is the number of nodes in the network
* `Ridx` is a vector indicating wind farm locations
* `T` is the number of time steps
* `Y` is the admittance matrix (n-by-n)
* `ref` is the index of the angle reference bus
* `k` is the vector of generator participation factors
"""
function fixed_wind_A(T,Y,ref,k)
    function ei(n,i)
        v = zeros(n)
        v[i] = 1.0
        return v
    end

    n = size(Y,1)

    # A has a block diagonal pattern where each
    # block is Atemp:
    Atemp = sparse([[  Y       -k];
                    ei(n,ref)'   0])

    # Now we can tile the Atemp matrix to generate A:
    A = Atemp
    for t = 2:T
        A = blkdiag(A, Atemp)
    end

    return full(A)
end

fixed_wind_A (generic function with 1 method)

In [7]:
?fixed_wind_A

search: 

```
fixed_wind_A(T,Y,ref,k) -> A
```

Generate the power balance constraint A matrix from problem dimensions, admittance matrix, and generator participation factors. Use this function when variable (renewable) injections are known, to run power flow and determine angle differences throughout the network.

Assumes the admittance matrix is n-by-n.

Returns A, which is (n+1)T-by-(nr+n+1)T

  * `nr` is the number of wind farms in the network
  * `n` is the number of nodes in the network
  * `Ridx` is a vector indicating wind farm locations
  * `T` is the number of time steps
  * `Y` is the admittance matrix (n-by-n)
  * `ref` is the index of the angle reference bus
  * `k` is the vector of generator participation factors

fixed_wind_A



In [14]:
"""
Solve the secular equation by returning `v` satisfying
s(v) = w'\*w = c,

where `w = num./(v - poles)`

Inputs:

* `num` is a diagonal matrix with numerator
* `poles` is a vector of poles (no need to sort them)
* `c` is the target constant; we return solutions s(v) = c.

Output:

* `vals` is a vector of solutions `v` satisfying s(v) = c
* `vecs` is a vector of vectors `w` corresponding to `vals`
"""
function solvesecular(
    num::Vector{Float64},
    poles::Vector{Float64},
    c::Float64
    )
    s(vv::Vector{Float64}) = [sumabs2(num./(v - poles)) for v in vv]
    s(v::Float64) = sumabs2(num./(v - poles))
    w(vv::Vector{Float64}) = [float(num./(v - poles)) for v in vv]
    w(v::Float64) = float(num./(v - poles))
    tiny = c/1e8

    vals = Vector{Float64}()
    sortedpoles = sort(poles)

    for i in 1:length(sortedpoles)
        upper = sortedpoles[i]

        # head left
        dist = finddist(i,sortedpoles,true)
        lower = upper - dist/2
        intersect = secularintersect(c,upper,lower,s,tiny)

        isinf(intersect) && continue

        # there must be at least one solution for leftmost pole
        if i == 1
            ex = 0
            while isnan(intersect)
                ex += 1
                if ex == 10
                    intersect = NaN
                    break
                end
                warn("expanding $ex")
                lower -= 100*dist
                intersect = secularintersect(c,upper,lower,s,tiny)
            end
        end
        push!(vals,intersect)

        # head right
        dist = finddist(i,sortedpoles,false)
        lower = upper + dist/2
        intersect = secularintersect(c,upper,lower,s,tiny)
        # there must be at least one solution for rightmost pole
        if i == length(sortedpoles)
            ex = 0
            while isnan(intersect)
                ex += 1
                if ex == 10
                    intersect = NaN
                    break
                end
                warn("expanding $ex")
                lower += 100*dist
                intersect = secularintersect(c,upper,lower,s,tiny)
            end
        end
        push!(vals,intersect)
    end
    filter!(x -> !isnan(x),vals)
    return vals, w(vals)
end

solvesecular (generic function with 1 method)

In [15]:
?solvesecular

search: 

Solve the secular equation by returning `v` satisfying s(v) = w'*w = c,

where `w = num./(v - poles)`

Inputs:

  * `num` is a diagonal matrix with numerator
  * `poles` is a vector of poles (no need to sort them)
  * `c` is the target constant; we return solutions s(v) = c.

Output:

  * `vals` is a vector of solutions `v` satisfying s(v) = c
  * `vecs` is a vector of vectors `w` corresponding to `vals`

solvesecular



In [3]:
?createY

search: 

```
createY(f,t,r,x,b) -> Y
```

Create an admittance matrix for AC power flow.

```
createY(f,t,x) -> Y
```

Create an admittance matrix for DC power flow.

createY searchsortedlast AbstractSparseMatrix



In [4]:
?mat2tmpinst

search: 

```
mat2tmpinst(name) -> d
```

Loads (and generates) everything needed to perform temporal instanton analysis for any network supported by [MatpowerCases.jl](https://github.com/kersulis/MatpowerCases.jl). `d` is an instance of `InstantonOutputData`.

mat2tmpinst



[hello][1]

[1]: https://github.com/kersulis/MatpowerCases.jl